https://note.com/scilabcafe/n/n41c75c5223f4

「移動平均線がゴールデンクロスしたときに買う」など、一定の売買ルールに従って取引を行った場合の収支を、過去のデータを使って検証することをバックテストといいます。

# １．株価データを取得する
下記を参考にOHLCV（始値 / 高値 / 安値 / 終値 / 出来高）形式のトヨタ株(7203.JP)データを取得します。データの取得期間は、2022年10月1日から現在の日付までです。

In [3]:
import pandas_datareader.data as web
import datetime

start = '2022-10-01'
end = datetime.date.today()
code = '7203.JP' # トヨタ

data = web.DataReader(code, 'stooq', start, end)

# 日付を昇順に並び替える
data.sort_index(inplace=True)

# ２．ライブラリをインポートする
まず、backtesting ライブラリから、Strategyクラス、Backtestクラス、backtesting.libからcrossoverクラス、backtesting.testからSMAクラスをインポートします。

- Strategyクラス
  - トレード戦略を実装するための基本クラスです。トレードのエントリーとエグジットの条件などを定義します。
  - 独自の戦略を作成する場合は、Strategyクラスをサブクラス化してカスタム戦略を実装します。

- Backtestクラス
  - バックテストの実行を管理するための主要なクラスです。
  - バックテストの期間、データ、戦略、手数料などのパラメータを設定し、実際のバックテストを実行します。

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from backtesting import Backtest

C:\Users\20222305\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:54: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# ３．売買ルールを作成する
バックテストを実行する際には、先にバックテストで使う売買のルールを定義します。

そのために、売買のルールを管理する Strategy クラスを継承したクラスを作成します。

ここでは、ゴールデンクロスで買い、デッドクロスで売るルールを定義しています。短期移動平均の設定日数は5日、長期移動平均の設定日数は25日としています。

In [19]:
class SmaCross(Strategy):
    ns = 5 # 短期移動平均日数
    nl = 25 # 長期移動平均日数

    def init(self):
        # 短期移動平均
        self.smaS = self.I(SMA, self.data['Close'], self.ns)
        # 長期移動平均
        self.smaL = self.I(SMA, self.data['Close'], self.nl)

    def next(self): # チャートデータの行ごとに呼び出される
        # smaS > smaL で買う
        if crossover(self.smaS, self.smaL):
            self.buy() # 買い
        # smaS < smaL で売る
        elif crossover(self.smaL, self.smaS):
            self.position.close() # 売り

# 4．バックテストを実行する

売買のルールを定義した後は、バックテストを行います。

バックテストの実行は、Backtest クラスで行います。

まずは Backtest クラスのインスタンスを生成し、run()メソッドで実行します。更に、plot()メソッドで実行結果をグラフ表示します。

In [21]:
# バックテストを設定。バックテストクラスを初期化してインスタンスを生成
bt = Backtest(
    data, # チャートデータ
    SmaCross, # 売買戦略
    # cash=1000, # 最初の所持金
    trade_on_close=True # True：現在の終値で取引，False：次の時間の始値で取引
)

# run()メソッドでバックテストを実行
result = bt.run()

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Start                     2022-10-03 00:00:00
End                       2025-02-27 00:00:00
Duration                    878 days 00:00:00
Exposure Time [%]                    56.89949
Equity Final [$]                     10196.29
Equity Peak [$]                      13518.12
Return [%]                             1.9629
Buy & Hold Return [%]                47.08213
Return (Ann.) [%]                       0.838
Volatility (Ann.) [%]                17.74422
CAGR [%]                              0.55948
Sharpe Ratio                          0.04723
Sortino Ratio                         0.06982
Calmar Ratio                          0.02894
Max. Drawdown [%]                   -28.95617
Avg. Drawdown [%]                    -6.07692
Max. Drawdown Duration      358 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                                   17
Win Rate [%]                         29.41176
Best Trade [%]                        25.5011
Worst Trade [%]                   

GridPlot(id='p2747', ...)

# Backtest.run() の出力項目とその意味

## **概要**
`Backtest.run()` は、バックテストの結果を `pandas.Series` の形式で返します。
この出力には、パフォーマンス指標やトレード統計が含まれます。

## **出力項目一覧**

| 項目名 | 説明 |
|---|---|
| **Start** | バックテストの開始日時 |
| **End** | バックテストの終了日時 |
| **Duration** | バックテストの期間 |
| **Exposure Time [%]** | ポジションを持っていた時間の割合 |
| **Equity Final [$]** | 最終的な資産残高 |
| **Equity Peak [$]** | 最大の資産残高 |
| **Return [%]** | 総リターン（%） |
| **Buy & Hold Return [%]** | 「買い持ち戦略」と比較したリターン |
| **Return (Ann.) [%]** | 年率リターン |
| **Volatility (Ann.) [%]** | 年率ボラティリティ |
| **Sharpe Ratio** | シャープレシオ（リターン / ボラティリティ） |
| **Sortino Ratio** | ソルティノレシオ（リスク調整後リターン） |
| **Calmar Ratio** | カルマーレシオ（リターン / 最大ドローダウン） |
| **Max. Drawdown [%]** | 最大ドローダウン |
| **Avg. Drawdown [%]** | 平均ドローダウン |
| **Max. Drawdown Duration** | 最大ドローダウン期間 |
| **Avg. Drawdown Duration** | 平均ドローダウン期間 |
| **# Trades** | 総トレード数 |
| **Win Rate [%]** | 勝率 |
| **Best Trade [%]** | 最大の利益トレード（%） |
| **Worst Trade [%]** | 最大の損失トレード（%） |
| **Avg. Trade [%]** | 平均トレードリターン |
| **Max. Trade Duration** | 最大トレード期間 |
| **Avg. Trade Duration** | 平均トレード期間 |
| **Profit Factor** | プロフィットファクター（総利益 / 総損失） |
| **Expectancy [%]** | 期待値（平均リターン） |
| **SQN** | システムクオリティナンバー（トレードの安定性指標） |

## **重要な指標の解釈**

- **`Return [%]`** → 総リターン（この戦略の収益率）
- **`Buy & Hold Return [%]`** → 最初に買って最後まで持ち続けた場合のリターン
- **`Sharpe Ratio`** → 値が **1.0以上** ならOK、**2.0以上** なら優れた戦略
- **`Max. Drawdown [%]`** → 最大の損失割合、これが小さいほど良い
- **`Win Rate [%]`** → 勝率、通常 **50%以上** で安定
- **`Profit Factor`** → **1.5以上** ならOK、**2.0以上** なら良い戦略
- **`SQN`** → トレードシステムの品質、**1.6以上** ならOK、**2.5以上** なら優秀

## **まとめ**
✅ `Backtest.run()` の出力には、パフォーマンス・リスク・トレード統計が含まれる  
✅ `Return [%]` や `Sharpe Ratio` で戦略のパフォーマンスを評価できる  
✅ `Profit Factor` や `SQN` を活用して戦略の安定性をチェック  

これを基に `Backtest.run()` の結果を分析し、最適なトレード戦略を設計しましょう！


# ５．パラメータを最適化する
バックテストの結果を使って、さらによい結果を出すためにパラメータの最適化を行います。ここでのパラメータの最適化とは、移動平均線の日数を最適にすることを指します。パラメータの最適化は、Backtest クラスの optimize メソッドで行います。

# ↓時間がかかってしまう

**maxmizeには、前項でバックテストを実行した結果の項目から最適化したい項目を指定します。**

指定しない場合は、SQNスコアが最適化されます。constraint では、売買のルールを定義したクラスが引数として渡されるので、lambda の形式で渡されたクラスのメンバを使って条件式を作成します。

In [ ]:
# # optimize()メソッドを実行して最適化
# result = bt.optimize(
#     ns=range(5, 25, 5),  # 短期移動平均日数
#     nl=range(5, 75, 5),  # 長期移動平均日数
#     maximize= 'Return [%]',  #※maximizeはデフォルトではSQN(System Quality Number)です
#     constraint=lambda r: r.ns < r.nl  # consraintオプションを付けることで短期SMAが長期SMAより長くなることを禁止できます
# )

# # 実行結果のデータを表示
# print(result)

# # plot()メソッドを実行して実行結果をグラフで表示
# bt.plot()

Backtest.optimize:   0%|          | 0/46 [00:00<?, ?it/s]

# RSIを用いた取引ルールでのバックテスト

# RSIを算出する関数を定義

**RSI（Relative Strength index：相対力指数）は、相場の過熱度をもとに「売られすぎている」「買われすぎている」という状態を数値化したもの** です。売られすぎか買われすぎかを判断するオシレーター系のテクニカル指標の１つです。

RSIでは、売られすぎ、買われすぎという状態を0～100の数値で表します。買われすぎのピークが近づくとRSIの値は100に近くなります。逆に、売られすぎのピークが近づくとRSIの値は0に近くなります。

一般的に、値が30を下回ると売られすぎ、70を超えると買われすぎの目安と言われています。

In [13]:
import talib as ta
def RSI(close, n1, n2):
    # RSIを算出
    rsiS = ta.RSI(close, timeperiod=n1)  # 短期 n1日
    rsiL = ta.RSI(close, timeperiod=n2)  # 長期 n2日
    return rsiS, rsiL

In [14]:
class RSICross(Strategy):
    ns = 14 # 短期
    nl = 28 # 長期

    def init(self):
        # rsiS / rsiL
        self.rsiS, self.rsiL = self.I(RSI, self.data['Close'], self.ns, self.nl)


    def next(self): # チャートデータの行ごとに呼び出される
        # rsiS > rsiL で買う
        if crossover(self.rsiS, self.rsiL):
            self.buy() # 買い
        # rsiS < rsiL で売る
        elif crossover(self.rsiL, self.rsiS):
            self.position.close() # 売り

In [15]:
# バックテストを設定。バックテストクラスを初期化してインスタンスを生成
bt = Backtest(
    data, # チャートデータ
    RSICross, # 売買戦略
    # cash=1000, # 最初の所持金
    trade_on_close=True # True：現在の終値で取引，False：次の時間の始値で取引
)

# run()メソッドでバックテストを実行
result = bt.run()

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Start                     2022-10-03 00:00:00
End                       2025-02-27 00:00:00
Duration                    878 days 00:00:00
Exposure Time [%]                    52.64055
Equity Final [$]                     14995.73
Equity Peak [$]                      17195.73
Return [%]                            49.9573
Buy & Hold Return [%]                48.34155
Return (Ann.) [%]                    18.99896
Volatility (Ann.) [%]                22.04926
CAGR [%]                             12.33252
Sharpe Ratio                          0.86166
Sortino Ratio                         1.57452
Calmar Ratio                            1.485
Max. Drawdown [%]                   -12.79387
Avg. Drawdown [%]                    -3.42783
Max. Drawdown Duration      294 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                   29
Win Rate [%]                         34.48276
Best Trade [%]                       37.00205
Worst Trade [%]                   

GridPlot(id='p1755', ...)

# ↓時間がかかってしまう

In [ ]:
# # optimize()メソッドを実行して最適化
# result = bt.optimize(
#     ns=range(5, 25, 5),  # 短期
#     nl=range(5, 75, 5),  # 長期
#     maximize= 'Return [%]',  #※maximizeはデフォルトではSQN(System Quality Number)です
#     constraint=lambda r: r.ns < r.nl  # consraintオプションを付けることで短期rsiが長期rsiより長くなることを禁止できます
# )

# # 実行結果のデータを表示
# print(result)

# # plot()メソッドを実行して実行結果をグラフで表示
# bt.plot()

# MACDを用いた取引ルールでのバックテスト

MACDを算出する関数を定義

In [16]:
def MACD(close, n1, n2, n3):
    # MACD、シグナル、ヒストグラムを算出
    macd, macdsignal, _ = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=n3)
    return macd, macdsignal

Strategyクラスを継承したMADCCrossクラスを定義

In [17]:
class MACDCross(Strategy):
    n1 = 12 # fastperiod
    n2 = 26 # slowperiod
    n3 = 9 # signalperiod

    def init(self):
        # macd, macdsignal
        self.macd, self.macdsignal = self.I(MACD, self.data['Close'], self.n1, self.n2, self.n3)


    def next(self): # チャートデータの行ごとに呼び出される
        # macd > macdsignal で買う
        if crossover(self.macd, self.macdsignal):
            self.buy() # 買い
        # macd < macdsignal で売る
        elif crossover(self.macdsignal, self.macd):
            self.position.close() # 売り

バックテストクラスのインスタンス化とバックテストの実行

In [18]:
# バックテストを設定。バックテストクラスを初期化してインスタンスを生成
bt = Backtest(
    data, # チャートデータ
    MACDCross, # 売買戦略
    # cash=1000, # 最初の所持金
    trade_on_close=True # True：現在の終値で取引，False：次の時間の始値で取引
)

# run()メソッドでバックテストを実行
result = bt.run()

# 実行結果のデータを表示
print(result)

# plot()メソッドを実行して実行結果をグラフで表示
bt.plot()

Start                     2022-10-03 00:00:00
End                       2025-02-27 00:00:00
Duration                    878 days 00:00:00
Exposure Time [%]                    49.57411
Equity Final [$]                     11393.31
Equity Peak [$]                      13873.57
Return [%]                            13.9331
Buy & Hold Return [%]                47.08213
Return (Ann.) [%]                     5.75962
Volatility (Ann.) [%]                18.25736
CAGR [%]                              3.81484
Sharpe Ratio                          0.31547
Sortino Ratio                         0.48799
Calmar Ratio                           0.2463
Max. Drawdown [%]                   -23.38446
Avg. Drawdown [%]                    -3.63667
Max. Drawdown Duration      358 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                   24
Win Rate [%]                             37.5
Best Trade [%]                       15.15167
Worst Trade [%]                   

GridPlot(id='p2166', ...)

# ↓時間がかかってしまう

パラメータの最適化

In [ ]:
# # optimize()メソッドを実行して最適化
# result = bt.optimize(
#     n1=range(5, 55, 5),  # fastperiod
#     n2=range(10, 75, 5),  # slowperiod
#     n3=range(10, 75, 5),  # signalperiod
#     maximize= 'Return [%]',  #※maximizeはデフォルトではSQN(System Quality Number)です
#     constraint=lambda r: r.n1 < r.n2  # consraintオプションを付けることでfastperiodがslowperiodより長くなることを禁止できます
# )

# # 実行結果のデータを表示
# print(result)

# # plot()メソッドを実行して実行結果をグラフで表示
# bt.plot()